In [2]:
# import packages

import os, sys
sys.path.append('../')
from pathlib import Path

# numbers
import numpy as np
import pandas as pd

import anndata


In [3]:
os.chdir('/home/coakes/rat/')

# Metadata Loading

In [4]:
combined_meta = pd.read_csv('rat_nid_metadata.csv')
combined_meta

,pid,viallabel,calculated.variables.pct_body_fat_change,calculated.variables.pct_body_lean_change,calculated.variables.pct_body_fluid_change,calculated.variables.lactate_change_dueto_train,calculated.variables.vo2_max_change,calculated.variables.coll_time_train,calculated.variables.deathtime_after_train,calculated.variables.frozetime_after_train,...,sex,tissue,time,rank,nid,barcode,path,nac_path,mat_path,amb_path
0,10023259,90217013001,1.8,6.9,1.6,7.7,-1.0,168866,169279,176100,...,M,BLOOD,0,A,M0A,SRR25251722,/tolva/rat_attack/rat/count_matrices_SRR252517...,/tolva/rat_attack/rat/count_matrices_SRR252517...,/tolva/rat_attack/rat/count_matrices_SRR252517...,/tolva/rat_attack/rat/count_matrices_SRR252517...
1,10023259,90217013104,1.8,6.9,1.6,7.7,-1.0,168866,169279,171453,...,M,PLASMA,0,A,M0A,NaN,NaN,NaN,NaN,NaN
2,10023259,90217013105,1.8,6.9,1.6,7.7,-1.0,168866,169279,171453,...,M,PLASMA,0,A,M0A,NaN,NaN,NaN,NaN,NaN
3,10023259,90217013106,1.8,6.9,1.6,7.7,-1.0,168866,169279,171453,...,M,PLASMA,0,A,M0A,NaN,NaN,NaN,NaN,NaN
4,10023259,90217013107,1.8,6.9,1.6,7.7,-1.0,168866,169279,171453,...,M,PLASMA,0,A,M0A,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,10971645,90587017012,NaN,NaN,NaN,NaN,NaN,180724,180787,180734,...,F,WAT-SC,2,N,F2N,NaN,NaN,NaN,NaN,NaN
6152,10971645,90587017013,NaN,NaN,NaN,NaN,NaN,180724,180787,180734,...,F,WAT-SC,2,N,F2N,NaN,NaN,NaN,NaN,NaN
6153,10971645,90587017014,NaN,NaN,NaN,NaN,NaN,180724,180787,180734,...,F,WAT-SC,2,N,F2N,NaN,NaN,NaN,NaN,NaN
6154,10971645,90587017702,NaN,NaN,NaN,NaN,NaN,133620,180787,133621,...,F,NaN,2,N,F2N,NaN,NaN,NaN,NaN,NaN


# Metadata annotation of Omic Assays

In [15]:
for omic in ['ATAC', 'ACETYL', 'PROT', 'PHOSPHO', 'UBIQ', 'METAB','IMMUNO']:
    print(omic)
    adata = anndata.read_h5ad(f'omics_adata/rat_{omic}.h5ad')
    adata.X = np.nan_to_num(adata.X, nan=0)

    # Metabolomics and Immunoassay require different, analysis-specific preprocessing due to each encompassing multiple assays
    if omic in ['METAB', 'IMMUNO']:
        adata.obs.pid = adata.obs.pid.astype('int')
        temp = pd.merge(adata.obs.reset_index(), combined_meta, on=['pid', 'tissue'], how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)').set_index('index')
        temp = temp.groupby(['pid', 'tissue']).first().reset_index().drop('viallabel',axis=1)
        adata.obs = pd.merge(adata.obs.reset_index(), temp, on=['pid', 'tissue'], how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)').drop('index',axis=1)
    else:
        adata.obs['viallabel'] = adata.obs.X.astype('int')
        adata.obs = pd.merge(adata.obs.reset_index(), combined_meta, on='viallabel', how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)').set_index('index')
    # add gene to feature annotations for ATAC and PROT
    if omic in ['ATAC']:
        features = pd.read_csv(f'{omic}_features.csv')
        adata.var = pd.merge(adata.var, features, left_on = 'X', right_on = 'feature_ID', how = 'left').drop('X',axis=1)
        adata.var.chrom = adata.var.chrom.astype('str')
        adata.var.feature_ID = adata.var.feature_ID.astype('str')
    elif omic in ['PROT']:
        features = pd.read_csv('feature_to_gene.csv')
        temp_var = pd.merge(adata.var, features, left_on = 'X', right_on = 'feature_ID', how = 'left').drop('X',axis=1)
        temp_var = temp_var.groupby('feature_ID').first().reset_index()
        adata.var = pd.merge(adata.var, temp_var, left_on = 'X', right_on = 'feature_ID', how = 'left').drop('X',axis=1)
        adata.var.feature_ID = adata.var.feature_ID.astype('str')
        adata.var.custom_annotation = adata.var.custom_annotation.astype('str')
        adata.var.kegg_id = adata.var.kegg_id.astype('str')
    # rename WAT for consistency 
    adata.obs = adata.obs.replace(['WAT-SC'], 'WAT', )
    # only keep samples with time metadata
    adata = adata[~adata.obs.time.isna()]
    adata.write(f'omics_adata/rat_{omic}_meta.h5ad')

ATAC


/tmp/ipykernel_1969272/3970661747.py:17: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  features = pd.read_csv(f'{omic}_features.csv')
/tmp/ipykernel_1969272/3970661747.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adata.obs = adata.obs.replace(['WAT-SC'], 'WAT', ).infer_objects(copy=False)
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", Implicit

ACETYL
PROT


/tmp/ipykernel_1969272/3970661747.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adata.obs = adata.obs.replace(['WAT-SC'], 'WAT', ).infer_objects(copy=False)
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as

PHOSPHO
UBIQ


/tmp/ipykernel_1969272/3970661747.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adata.obs = adata.obs.replace(['WAT-SC'], 'WAT', ).infer_objects(copy=False)
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as

METAB


/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribu

IMMUNO


/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribu

# Methyl Data Loading

Extra Processing Required because R memory limits did not allow for the merging of the Methyl dataset

In [5]:
meth_adata_1 = anndata.read_h5ad('omics_adata/rat_METHYL_1.h5ad')
meth_adata_2 = anndata.read_h5ad('omics_adata/rat_METHYL_2.h5ad')
meth_adata_3 = anndata.read_h5ad('omics_adata/rat_METHYL_3.h5ad')
all_present = list(set(meth_adata_1.var.X) & set(meth_adata_2.var.X) & set(meth_adata_3.var.X))
filt_1 = meth_adata_1[:,meth_adata_1.var.X.isin(all_present)].copy()
filt_1.var.index = filt_1.var.X
filt_2 = meth_adata_2[:,meth_adata_2.var.X.isin(all_present)].copy()
filt_2.var.index = filt_2.var.X
filt_3 = meth_adata_3[:,meth_adata_3.var.X.isin(all_present)].copy()
filt_3.var.index = filt_3.var.X
inner_adata = anndata.concat([filt_1, filt_2, filt_3], axis = 0)
inner_adata

AnnData object with n_obs × n_vars = 400 × 1023169
    obs: 'X'

In [6]:
ufilt_1 = meth_adata_1[:,~meth_adata_1.var.X.isin(all_present)].copy()
ufilt_1.var.index = ufilt_1.var.X
ufilt_2 = meth_adata_2[:,~meth_adata_2.var.X.isin(all_present)].copy()
ufilt_2.var.index = ufilt_2.var.X
ufilt_3 = meth_adata_3[:,~meth_adata_3.var.X.isin(all_present)].copy()
ufilt_3.var.index = ufilt_3.var.X
outer_adata = anndata.concat([ufilt_1, ufilt_2, ufilt_3], axis = 0, join = 'outer')
outer_adata

AnnData object with n_obs × n_vars = 400 × 566365
    obs: 'X'

In [7]:
meth_adata = anndata.concat([inner_adata, outer_adata], axis = 1, join = 'outer')
meth_adata.obs['viallabel'] = meth_adata.obs.index.astype('int')
meth_adata.layers['counts'] = meth_adata.X
meth_adata

AnnData object with n_obs × n_vars = 400 × 1589534
    obs: 'viallabel'
    layers: 'counts'

In [8]:
meth_adata.obs = pd.merge(meth_adata.obs.reset_index(), combined_meta, on='viallabel', how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)').set_index('index')
meth_adata.obs = meth_adata.obs.replace(['WAT-SC'], 'WAT')
meth_adata

/tmp/ipykernel_1860286/1291696852.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  meth_adata.obs = meth_adata.obs.replace(['WAT-SC'], 'WAT')


AnnData object with n_obs × n_vars = 400 × 1589534
    obs: 'viallabel', 'pid', 'calculated.variables.pct_body_fat_change', 'calculated.variables.pct_body_lean_change', 'calculated.variables.pct_body_fluid_change', 'calculated.variables.lactate_change_dueto_train', 'calculated.variables.vo2_max_change', 'calculated.variables.coll_time_train', 'calculated.variables.deathtime_after_train', 'calculated.variables.frozetime_after_train', 'group', 'sex', 'tissue', 'time', 'rank', 'nid', 'barcode', 'path', 'nac_path', 'mat_path', 'amb_path'
    layers: 'counts'

In [9]:
meth_features = pd.read_csv('methyl_features.csv')
meth_features = meth_features.groupby('feature_ID').first().drop('tissue', axis=1).reset_index()
meth_adata.var = pd.merge(meth_adata.var, meth_features, left_on = 'X', right_on = 'feature_ID', how = 'left')
# connect features to genes
features = pd.read_csv('feature_to_gene.csv')
temp_var = pd.merge(meth_adata.var, features, left_on = 'feature_ID', right_on = 'feature_ID', how = 'left')
temp_var = temp_var.groupby('feature_ID').first().reset_index().drop('kegg_id',axis=1)
meth_adata.var = temp_var
meth_adata.var.head()

/tmp/ipykernel_1860286/4013687915.py:5: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  features = pd.read_csv('feature_to_gene.csv')


,feature_ID,Chr,Locus,EntrezID,Symbol,Strand,Width,NumSites,Sites,LocStart,LocEnd,entrez_gene,rgd_gene,gene_symbol,old_gene_symbol,ensembl_gene,relationship_to_gene,custom_annotation
0,chr1-100093,1,50046723-50046723,56816,Prkn,-,0,1,50046723,50046723,50046723,NaN,15009540.0,AABR07001519.1,None,ENSRNOG00000055547,202176.0,Distal Intergenic
1,chr1-100094,1,50046775-50046775,56816,Prkn,-,0,1,50046775,50046775,50046775,NaN,15009540.0,AABR07001519.1,None,ENSRNOG00000055547,202228.0,Distal Intergenic
2,chr1-10020,1,5010178-5010178,24414,Grm1,-,0,1,5010178,5010178,5010178,24414.0,2742.0,Grm1,Gprc1a;mGluR1,ENSRNOG00000014290,0.0,Intron
3,chr1-10021,1,5010265-5010265,24414,Grm1,-,0,1,5010265,5010265,5010265,24414.0,2742.0,Grm1,Gprc1a;mGluR1,ENSRNOG00000014290,0.0,Intron
4,chr1-100227_cluster1,1,50113565-50113565,499021,Pacrg,+,0,1,50113565,50113565,50113565,NaN,15009540.0,AABR07001519.1,None,ENSRNOG00000055547,269018.0,Distal Intergenic


In [10]:
# drop columns missing data
meth_adata = meth_adata[:,~np.isnan(meth_adata.X).any(axis=0)]
meth_adata

View of AnnData object with n_obs × n_vars = 400 × 1023169
    obs: 'viallabel', 'pid', 'calculated.variables.pct_body_fat_change', 'calculated.variables.pct_body_lean_change', 'calculated.variables.pct_body_fluid_change', 'calculated.variables.lactate_change_dueto_train', 'calculated.variables.vo2_max_change', 'calculated.variables.coll_time_train', 'calculated.variables.deathtime_after_train', 'calculated.variables.frozetime_after_train', 'group', 'sex', 'tissue', 'time', 'rank', 'nid', 'barcode', 'path', 'nac_path', 'mat_path', 'amb_path'
    var: 'feature_ID', 'Chr', 'Locus', 'EntrezID', 'Symbol', 'Strand', 'Width', 'NumSites', 'Sites', 'LocStart', 'LocEnd', 'entrez_gene', 'rgd_gene', 'gene_symbol', 'old_gene_symbol', 'ensembl_gene', 'relationship_to_gene', 'custom_annotation'
    layers: 'counts'

In [11]:
meth_adata.write('omics_adata/rat_METHYL_meta.h5ad')
meth_adata

/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/coakes/miniconda3/envs/rata/lib/python3.12/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribu

AnnData object with n_obs × n_vars = 400 × 1023169
    obs: 'viallabel', 'pid', 'calculated.variables.pct_body_fat_change', 'calculated.variables.pct_body_lean_change', 'calculated.variables.pct_body_fluid_change', 'calculated.variables.lactate_change_dueto_train', 'calculated.variables.vo2_max_change', 'calculated.variables.coll_time_train', 'calculated.variables.deathtime_after_train', 'calculated.variables.frozetime_after_train', 'group', 'sex', 'tissue', 'time', 'rank', 'nid', 'barcode', 'path', 'nac_path', 'mat_path', 'amb_path'
    var: 'feature_ID', 'Chr', 'Locus', 'EntrezID', 'Symbol', 'Strand', 'Width', 'NumSites', 'Sites', 'LocStart', 'LocEnd', 'entrez_gene', 'rgd_gene', 'gene_symbol', 'old_gene_symbol', 'ensembl_gene', 'relationship_to_gene', 'custom_annotation'
    layers: 'counts'